In [318]:
# Import Dependencies
import os
import pandas as pd
from bs4 import BeautifulSoup as bs
from splinter import Browser
from selenium import webdriver
import requests
import time

In [319]:
# Use requests and BeautifulSoup to scrape Nasa News for latest news
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
response = requests.get(url)
soup = bs(response.text, 'lxml')
# print(soup.prettify())

In [320]:
results = soup.find('div', class_='features')
# for result in results:
#     print(result)

In [321]:
news_title = results.find('div', class_='content_title').text
newsp = results.find('div', class_='rollover_description').text
print(news_title)
print(newsp)



NASA Invests in Visionary Technology 




NASA is investing in technology concepts, including several from JPL, that may one day be used for future space exploration missions.







In [322]:
# Find image url for current featured mars image
executable_path = {'executable_path': '/Users/venessayeh/Downloads/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

nasa_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(nasa_url)

nasa_html = browser.html
nasa_soup = bs(nasa_html, "lxml")

In [323]:
featured = nasa_soup.find('div', class_='default floating_text_area ms-layer')
featured_image = featured.find('footer')
featured_image_url = 'https://www.jpl.nasa.gov'+ featured_image.find('a')['data-fancybox-href']
print(str(featured_image_url))

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA18328_ip.jpg


In [324]:
# Scrape Mars Weather Twitter account for latest weather report on Mars
twitter_url = 'https://twitter.com/marswxreport?lang=en'
twitter_response = requests.get(twitter_url)
twitter_soup = bs(twitter_response.text, 'lxml')
twitter_result = twitter_soup.find('div', class_='js-tweet-text-container')
# for tweet in twitter_result:
#     print(tweet)

In [325]:
mars_weather = twitter_result.find('p', class_='js-tweet-text').text
mars_weather

'Sol 2030 (April 22, 2018), Sunny, high -4C/24F, low -73C/-99F, pressure at 7.21 hPa, daylight 05:25-17:21'

In [326]:
# Scrape space-facts.com for mars fact using Pandas read_html function
mars_facts_url = 'https://space-facts.com/mars/'
tables = pd.read_html(mars_facts_url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [327]:
# Create pandas dataframe
df = tables[0]
df.columns = ['Description', 'Value']
df.head()

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"


In [328]:
# Reset index 
df.set_index('Description', inplace=True)
df.head()

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"


In [329]:
# Export pandas df to html script
mars_facts = df.to_html()
mars_facts.replace("\n", "")
df.to_html('mars_facts.html')

In [331]:
# Scrape astrogeology.usgs.gov for hemisphere image urls and titles
hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemisphere_url)

hemisphere_html = browser.html
hemisphere_soup = bs(hemisphere_html, 'lxml')
base_url ="https://astrogeology.usgs.gov"

image_list = hemisphere_soup.find_all('div', class_='item')

# Create list to store dictionaries of data
hemisphere_image_urls = []

# Loop through each hemisphere and click on link to find large resolution image url
for image in image_list:
    hemisphere_dict = {}
    
    href = image.find('a', class_='itemLink product-item')
    link = base_url + href['href']
    browser.visit(link)
    
    time.sleep(1)
    
    hemisphere_html2 = browser.html
    hemisphere_soup2 = bs(hemisphere_html2, 'lxml')
    
    img_title = hemisphere_soup2.find('div', class_='content').find('h2', class_='title').text
    hemisphere_dict['title'] = img_title
    
    img_url = hemisphere_soup2.find('div', class_='downloads').find('a')['href']
    hemisphere_dict['url_img'] = img_url
    
    # Append dictionary to list
    hemisphere_image_urls.append(hemisphere_dict)
      
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'url_img': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'url_img': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'url_img': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'url_img': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]